# KAIST AI599 법률인공지능 Assignment 2
- AI Part: 60 Points
- Law Part: 40 Points
- 총점: 100 Points

이름: 강여진, 김동규, 김학성, 김형, 여인경

## AI Part
이번 과제에서는 사실관계가 주어졌을 때 적용 법률(statute)을 예측하는 모델을 만들어 봅니다. 먼저 아래와 같이 데이터를 다운로드 받습니다.

In [ ]:
!pip install -q datasets==2.4.0

In [ ]:
import datasets

dataset_card = "lbox/lbox_open"
task = "statute_classification"
data = datasets.load_dataset(dataset_card, task)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


  0%|          | 0/4 [00:00<?, ?it/s]

데이터를 개별 확인해 보면, 아래와 같은 형태를 띄고 있는 것을 볼 수 있습니다.

In [ ]:
data_v = data["validation"]
data_v[40]

{'id': 1286,
 'casetype': 'criminal',
 'casename': '교통사고처리특례법위반(치사)',
 'statutes': ['교통사고 처리특례법 제3조 제1항', '형법 제268조'],
 'facts': '피고인은 (차량번호 1 생략) 케이(K)5 택시의 운전업무에 종사하는 사람이다.\n피고인은 2021. 5. 19. 03:40경 위 택시를 운전하여 춘천시 B 앞 편도 2차로 도로를 퇴계사거리 방면에서 석사사거리 방면으로 시속 약 48킬로미터로 진행하던 중 전방 및 좌우 주시의무를 게을리 하고 조향 및 제동 장치를 제때 조작하지 못한 업무상 과실로 마침 전방 횡단보도 신호가 적색임에도 피고인 진행방향 좌측에서 우측으로 위 도로를 무단횡단 하던 피해자 C(남, 72세)을 미처 발견하지 못하고 피고인 택시의 앞 범퍼 부분으로 피해자를 들이받아 그 충격으로 피해자로 하여금 2021. 5. 21. 21:40경 춘천시 D 소재 E병원에서 다발성장기부전 등으로 사망에 이르게 하였다.'}

여기서 저희는 사실관계(facts)를 입력값으로 하여 적용법률(statutes)를 예측하고자 합니다. 적용법률이 여러개일 수 있기 때문에, `list` 형태인 점을 주의하세요.

**Problem AI.1** *(10 points)*. `statutes`는 text string으로 돼 있습니다. 분류문제로 치환하기 위해서는, 먼저 train 시에 관측되는 법률의 가지수를 찾으세요 *(5 points)*. 또한 validation에서 관측되는 볍률중 학습시에 본적이 없는 법률이 있는데요, (법률 단위가 아닌) 예제단위로 이 비율을 계산하세요 *(5 points)*. 

In [ ]:
data_t = data['train']
statutes = sorted(set.union(*map(set, data_t['statutes'])))
#print(statutes)
print(len(statutes))

151


In [ ]:
cnt = 0
for d in data_v:
    cnt += not set(d['statutes']).issubset(statutes)
print(cnt / len(data_v))

0.018115942028985508


**Problem AI.2** *(30 points)*. Hugging Face의 `multilingual-bert-cased`을 활용해 사실관계->법률분류 모델을 학습하고 validation 데이터에 대해 정확성을 보고하세요. 여러개의 적용 법률중 하나라도 맞추면 정답으로 간주합니다. 학습 때 못 본 법률을 고려해, "Unknown"에 해당하는 분류를 추가하고 학습 때 고려할 수 있도록 하세요 (다만, 학습 때 못 본 법률의 경우 "Unknown"으로 매핑되면 오답으로 간주). Colab 기준 학습 30분 이내에 70% 이상의 정확성이 나올 수 있도록 합니다. 수업 중 공개한 예제코드를 활용하셔도 좋습니다. 

In [ ]:
!pip install --quiet transformers==4.22.0
!pip install --quiet sentencepiece==0.1.97
!pip install --quiet pytorch-lightning==1.7.6

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

from functools import reduce
import random

import torch
import numpy as np
import transformers
import pytorch_lightning as pl
from transformers import AutoTokenizer, BertForSequenceClassification

import pandas as pd

In [ ]:
class StatutesDataModule(pl.LightningDataModule):
    def __init__(self, statutes, tokenizer, data, batch_size=16, max_input_len=512):
        super().__init__()
        self.statutes = statutes
        self.tokenizer = tokenizer
        self.data = data
        self.batch_size = batch_size
        self.max_input_len = max_input_len 
    
    def setup(self, stage):
        pass
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.data['train'], batch_size=self.batch_size, shuffle=True, collate_fn=self._collate_fn)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.data['validation'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.data['test'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)
    
    def _collate_fn(self, batch):
        inputs = self.tokenizer([x['facts'] for x in batch], max_length=self.max_input_len, padding=True, truncation=True, return_tensors='pt')
        labels = [x['statutes'] for x in batch]

        return inputs, labels

In [ ]:
class StatutesClassifier(pl.LightningModule):
    def __init__(self, statutes, backbone, learning_rate=1e-5, mode='multi-class'):
        super().__init__()
        self.statutes = statutes
        self.backbone = backbone
        self.learning_rate = learning_rate
        self.mode = mode
        self.criterion = torch.nn.CrossEntropyLoss() if mode == 'multi-class' else torch.nn.BCEWithLogitsLoss()
    
    def forward(self, batch):
        inputs, labels = batch
        logits = self.backbone(**inputs).logits
        targets = torch.zeros_like(logits)
        for i, label in enumerate(labels):
            label_id = [(self.statutes.index(x) if x in self.statutes else 0) for x in label]
            targets[i, label_id] = 1 / len(label_id) if self.mode == 'multi-class' else 1
        loss = self.criterion(logits, targets)

        return loss, logits

    def training_step(self, batch, batch_idx):
        loss, _ = self.forward(batch)
        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        return self._evaluation_step(batch) 
    
    def validation_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs)

    def test_step(self, batch, batch_idx):
        return self._evaluation_step(batch) 

    def test_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs, True)
   
    def _evaluation_step(self, batch):
        loss, logits = self.forward(batch)
        _, gts = batch
        if self.mode == 'multi-class':
            prs = np.array(self.statutes)[logits.argmax(-1).cpu()].tolist()
        else:
            prs = [np.array(self.statutes)[np.where(logit.sigmoid().cpu() > 0.5)[0]] for logit in logits]

        return {'loss': loss.item(), 'gts': gts, 'prs': prs}

    def _evaluation_epoch_end(self, outputs, test=False):
        avg_loss = np.mean([x['loss'] for x in outputs])
        gts = reduce(lambda x,y: x + y, [x['gts'] for x in outputs], [])
        prs = reduce(lambda x,y: x + y, [x['prs'] for x in outputs], [])
        if self.mode == 'multi-class':
            acc = sum([(pr in gt) for gt, pr in zip(gts, prs)]) / len(gts)
        else:
            acc = sum([(set(pr) == set(gt)) for gt, pr in zip(gts, prs)]) / len(gts)
        
        print('='*50)
        print(f'avg_loss: {avg_loss}')
        print(f'ACC: {acc}')
        
        target_ids = random.sample(range(len(gts)), 2)
        for target_id in target_ids:
            print(f'GT: {gts[target_id]}\t\t\tPR: {prs[target_id]}')
        
        if self.mode == 'multi-class' and test:
            df = pd.DataFrame(columns=['idx', 'gt', 'pr', 'casetype', 'casename', 'facts'])
            for i, (gt, pr) in enumerate(zip(gts, prs)):
                if pr not in gt:
                    df = df.append(pd.DataFrame({'idx': [i], 'gt': ', '.join(gt), 'pr': pr, 
                                                 'casetype': data['test'][i]['casetype'], 
                                                 'casename': data['test'][i]['casename'], 
                                                 'facts': data['test'][i]['facts']}), ignore_index=True)
            df.to_csv('GT_vs_PR.csv', index=False)
            

    def configure_optimizers(self):
        grouped_params = [{'params': list(filter(lambda p: p.requires_grad, self.parameters())), 'lr': self.learning_rate}]
        optimizer = torch.optim.AdamW(grouped_params, lr=self.learning_rate)

        return {'optimizer': optimizer}

In [10]:
backbone_card = 'bert-base-multilingual-cased'

# data
statutes = ['Unknown'] + sorted(set.union(*map(set, data['train']['statutes'])))
tokenizer = AutoTokenizer.from_pretrained(backbone_card)
data_module = StatutesDataModule(statutes, tokenizer, data, batch_size=24, max_input_len=256)

# model
backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(statutes))
model = StatutesClassifier(statutes, backbone, learning_rate=1e-5, mode='multi-class')

# trainer
n_gpus = 1 #torch.cuda.device_count()
trainer = pl.Trainer(max_epochs=10, gpus=n_gpus, fast_dev_run=not True)

trainer.fit(model, data_module)
trainer.test(model, data_module)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


avg_loss: 5.085952281951904
ACC: 0.0
GT: ['도로교통법 제148조의2 제1항', '도로교통법 제152조 제1호', '도로교통법 제43조', '도로교통법 제44조 제1항']			PR: 형법 제268조.
GT: ['형법 제136조 제1항']			PR: 형법 제268조.


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


...

avg_loss: 1.2073314984639485
ACC: 0.894927536231884
GT: ['형법 제260조 제1항']			PR: 형법 제260조 제1항
GT: ['형법 제231조', '형법 제234조']			PR: 형법 제231조


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing: 0it [00:00, ?it/s]

avg_loss: 1.3397086064020793
ACC: 0.8731884057971014
GT: ['형법 제231조', '형법 제234조']			PR: 형법 제355조 제1항
GT: ['도로교통법 제152조 제1호', '도로교통법 제43조']			PR: 도로교통법 제43조


[{}]

**Problem AI.3** *(20 points)*. 2번 문제가 일반 분류 문제였다면, multi-label classification으로 접근해서 학습하고 정확성을 보고하세요. 각 예제마다 모든 적용 법률을 맞췄을 때만 정답으로 간주합니다. 

In [11]:
# model
backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(statutes))
model = StatutesClassifier(statutes, backbone, learning_rate=2e-5, mode='multi-label')

# trainer
n_gpus = 1 #torch.cuda.device_count()
trainer = pl.Trainer(max_epochs=50, gpus=n_gpus, fast_dev_run=not True)
trainer.fit(model, data_module)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


avg_loss: 0.701230376958847
ACC: 0.0
GT: ['공직선거법 제250조 제1항']			PR: ['공직선거법 제106조 제1항' '공직선거법 제113조 제1항' '공직선거법 제166조의2 제1항' '공직선거법 제167조 제3항'
 '공직선거법 제241조 제1항' '공직선거법 제250조 제2항' '공직선거법 제254조 제2항'
 '공직선거법 제255조 제1항 제2호' '공직선거법 제255조 제2항 제3호' '공직선거법 제255조 제2항 제4호'
 '공직선거법 제255조 제2항 제5호' '공직선거법 제57조의3 제1항' '공직선거법 제60조 제1항 제4호'
 '공직선거법 제67조 제3항' '공직선거법 제85조 제3항' '공직선거법 제87조 제1항 제3호'
 '공직선거법 제90조 제1항 제1호' '공직선거법 제91조 제1항' '공직선거법 제93조 제1항' '공직선거법 제95조 제1항'
 '공직선거법 제96조 제2항 제1호' '교통사고 처리특례법 제3조 제1항' '교통사고 처리특례법 제3조 제2항'
 '교통사고 처리특례법 제3조 제2항 제11호' '교통사고 처리특례법 제3조 제2항 제1호'
 '도로교통법 제148조의2 제3항 제1호' '도로교통법 제148조의2 제3항 제2호' '도로교통법 제148조의2 제3항 제3호'
 '도로교통법 제152조 제1호' '도로교통법 제154조 제2호' '도로교통법 제44조 제2항' '병역법 제69조 제1항'
 '병역법 제84조 제2항' '병역법 제87조 제3항' '병역법 제88조 제1항' '병역법 제88조 제1항 제1호'
 '아동복지법 제17조 제1항 제2호' '아동복지법 제17조 제1항 제3호' '아동복지법 제17조 제5호'
 '아동복지법 제71조 제1항 제2호' '아동복지법 제74조' '전기통신공사업법 제30조' '전기통신공사업법 제95조의2 제2호'
 '전기통신공사업법 제97조 제7호' '전자금융거래법 제6조 제3항 제2호' '전자금융거래법 제6조 제3항 제3호'
 '정보통신망 이용 촉진 및 정보보호 등에

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


...

Validation: 0it [00:00, ?it/s]

avg_loss: 0.011480262968689203
ACC: 0.7681159420289855
GT: ['형법 제245조']			PR: ['형법 제245조']
GT: ['형법 제307조 제2항']			PR: ['형법 제307조 제2항']


Validation: 0it [00:00, ?it/s]

avg_loss: 0.01194009130510191
ACC: 0.7934782608695652
GT: ['형법 제260조 제1항', '형법 제261조']			PR: ['형법 제260조 제1항' '형법 제261조']
GT: ['형법 제355조 제1항']			PR: ['형법 제347조 제1항']


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


## Law Part
위 모델을 활용하여 사실관계->법률 예측 모델의 분석 및 기획을 진행합니다.

**Problem Law.1** *(20 points)*. AI.2 모델의 출력값과 정답이 다른 경우는 주로 어떤 패턴이 있나요? 2가지의 패턴을 찾고 이유를 분석하세요. 

1. **범죄 사실(facts)이 긴 경우**
    1. 오답 빈도수가 가장 높은 죄명은 협박죄, 허위사실 적시 사이버 명예훼손죄, 허위사실 적시 명예훼손죄 등임. 해당 죄들은 구체적인 발언 내용, 행위 내용을 범죄사실에 명시해야 하기 때문에 범죄사실이 비교적 길어짐.
        
        ![picture](https://drive.google.com/uc?id=1OuFlmwovqw5Um3mCBQHLhzroAATHEKiR)
        
    2. 모델은 facts에서 주목해야 하는 주요 키워드에 적절한 attention을 주지 않고 facts 전체를 동일한 weight로 학습함. 이 경우 facts 내 전반적인 context에만 집중해 답을 내게 되어, 오답 발생함. 주요 키워드로는 다음과 같은 내용이 있음.
        1. **사이버 상에서 발생했는지? (ex. ‘카톡방’, ‘메시지’, ‘인터넷 커뮤니티’…)** → 이 경우 형법이 아니라 정통망법이 적용됨
        2. **적시한 내용이 ‘사실’인지 ‘허위사실’인지?** → 사실적시 명예훼손죄와 허위사실적시 명예훼손죄를 가르게 됨
        3. **발언 내용이 ‘모욕’인지 협박인지? (ex. 이로써 피고인은 ~ 모욕하였다.)** → 모욕죄와 명예훼손죄를 가르게 됨
        4. **발언 내용이 ‘성’과 관련된 내용인지?** → 모욕, 협박죄와 성폭법 위반 통매음을 가르게 됨
        등이 있음.
    3. 예를 들어, 모델이 **정통망법 위반 허위사실적시 사이버 명예훼손죄를 협박죄로 잘못 판단**한 경우,
        1. GT: ['정보통신망 이용 촉진 및 정보보호 등에 관한 법률 제70조 제2항'] 
        PR: 형법 제283조 제1항
        [{'id': 8292, 'casetype': 'criminal', 'casename': '정보통신망이용촉진및정보보호등에관한법률위반(명예훼손)', 'statutes': ['정보통신망 이용 촉진 및 정보보호 등에 관한 법률 제70조 제2항'], 'facts': '피고인과 피해자 B, 피해자 C은 D 주식회사에서 근무하는 버스기사로 직장 동료이다.\n1. 2020. 11. 5.자 범행\n피고인은 2020. 11. 5. 21:19 무렵 알 수 없는 곳에서 휴대전화를 이용해 D 주식회사 E번 버스를 운행하는 기사 49명이 참여하는 카카오톡 단체 채팅방에서「B 씨한테 전화하면 고소당하는 건가요?? 그렇다면! 무슨 이유인가요?? B 씨한테 전화해서 고소당하신 분 계신가요?? 그러다보니 이상한 생각이 들더라고요?? 둘의 관계가 뭐지?? 그냥 동료라면 C이가 나한테 절대 할 수도 없는... 해서는 안 되는 말인데....!?!?!? 그렇다고 피를 나눈 남매도 아니고...... 도대체 왜? C이가 저런 말을 했을까?? 머리가 깨질 것 같네요&#126;!!! 이것도 저것도 아니면....??? 혹시???? 그렇고 그런 사이????? 에이&#126;!!!! 설마&#126;!!!!!! 설마가 사람 잡는다고 하긴 하던데.... 아이고!! 머리가 더 깨질 것 같습니다&#126;!!!!」라고 말하였다.\n이로써 피고인은 피해자들을 비방할 목적으로 정보통신망을 통하여 피해자들이 불륜관계에 있다는 취지의 허위사실을 공공연하게 게시하여 피해자들의 명예를 훼손하였다.\n2. 2020. 11. 26.자 범행\n피고인은 2020. 11. 26. 14:05 무렵 알 수 없는 곳에서 휴대전화를 이용해 D 주식회사 E번 버스를 운행하는 기사 49명이 참여하는 카카오톡 단체 채팅방에서「그리고 둘이 그렇고 그런 사이.... 이 말도 성희롱인가요? 없는 말한 것도 아닌데 어느 누가 너! 누구한테 전화하지 마라 고소당하기 싫으면... 이런 소리 들으면 고민 안 하겠습니까?? 고소당하게 생겼는데. 별의별 생각이 다 들죠~?!?!."라는 메시지를 게시하였다.\n이로써 피고인은 피해자들을 비방할 목적으로 정보통신망을 통하여 피해자들이 불륜관계에 있다는 취지의 허위사실을 공공연하게 게시하여 피해자들의 명예를 훼손하였다.'}]
        2. ”이로써 피고인은~”에 해당하는 문장, “허위사실” 등이 허위사실 적시 사이버 명예훼손죄의 주요 키워드이지만, “어느 누가 너! 누구한테 전화하지 마라 고소당하기 싫으면…” 같은 전반적인 context를 보고 협박죄로 판단한 것으로 보임.
2. **‘죄수 관계’, ‘형법-특별법, 행정법 간의 관계’ 등 법률 각 조항 간 관계성을 누락한 경우**
    1. 죄수 - 공범 규정
        1. 피고인이 공동정범, 방조범에 해당하는 경우에는 형법 제30조 등 공범에 관한 규정이 적용되어야 함.
            
            ![picture](https://drive.google.com/uc?id=1utfn7Bhdxck1IqR0xXtlieMz9id80jga)
            
        2. 하지만 모델은 공범에 관한 법률 내용을 모름. 따라서 범죄사실에 “피고인은 ~~와 공모하여”라는 ‘공범성’을 명시하는 표현이 들어가더라도 공범 규정을 답하지 않는 오류가 발생함.
        3. 과실범, 부작위범, 미수 등에 대해서도 같은 문제가 발생할 것으로 보임.
    2. 죄수 - 상상적 경합
        1. 상상적 경합이란, 한 가지 행위가 수죄에 해당하는 경우를 말함.
        예를 들어 공무집행방해죄와 업무방해죄가 이에 해당함. 판례에 따르면 이는 한 가지 행위가 수 죄에 해당하는 ‘상상적 경합’ 관계임. 
            
            ![picture](https://drive.google.com/uc?id=1ulQFZylhp5uK4p9Z0aiLe4pgYtjC8jGh)
            
        2. 모델은 수죄 사이의 상상적 경합, 실체적 경합 관계도 모름. 따라서 “공무집행 방해죄, 업무방해죄”가 아니라 “업무방해죄” 한 가지 답만을 내놓았음.
    3. 형법-특별법 간 관계성
        1. 다음과 같이 형법에 우선하여 특별법이 적용되어야 하는 경우가 있음.
            1. 특가법(특정범죄 가중처벌 등에 관한 법률)
            1억원 이상의 뇌물죄 가중처벌, 상습 강도 등 가중처벌, 도주치사상의 뺑소니 가중처벌, 위험운전 치사상 등
            2. 폭처법(폭력행위 등 처벌에 관한 법률)
            집단 폭행, 집단 협박, 집단 재물손괴, 집단 상해 등 집단, 상습 폭력범 처벌 등
            3. 성폭법(성폭력범죄의 처벌 등에 관한 특례법)
            특수강도강간(주거침입+강간), 합동강간 등
            4. 아청법(아동, 청소년의 성 보호에 관한 법률)
            아동을 대상으로 하는 강간, 강제추행 등
            5. 교특법(교통사고 처리 등에 관한 특례법)
            6. 정통망법(정보통신망 이용촉진 및 정보 보호 등에 관한 법률)
            명예훼손이 사이버 상에서 이뤄진 경우
        2. 모델은 형법-특별법 간의 관계성을 모르기 때문에 다음과 같은 오류가 발생했음.
            
            ![picture](https://drive.google.com/uc?id=149MJcnoXVVOM-UdeTLpIVk1CmAeAJAPd)
            
            1. “운전업무에 종사하는 사람” 즉 택시 운전사가 교통사고를 낸 경우, 형법 제268조 업무상 과실치사상죄 뿐만 아니라 교특법 제3조 제1항이 함께 적용되어야 하나, 형법 제268조만 답함.
            2. 버스 등 운송수단의 운전자를 폭행한 경우 형법 제260조 제1항의 단순 폭행죄가 아니라 특가법 제5조의10 제1항이 적용되어야 하나, 형법 제260조 제1항만 답함.
    4. 행정법의 의무규정-벌칙규정 관계
        1. 아동복지법, 공직선거법, 전자금융거래법, 병역법, 도로교통법 등은 행정법임. 행정법은 수범자의 의무를 규정하는 “의무 규정”과 이를 위반했을 때의 처벌 내용을 규정하는 “벌칙 규정”으로 구성되어 있음.
        2. 하지만 모델은 각 행정법의 의무규정, 벌칙규정 쌍(pairs)의 관계를 모르기 때문에 “벌칙 규정”만 명시하는 일이 발생하는 것으로 보임.
            
            ![picture](https://drive.google.com/uc?id=1yI1-Lj3-DaCsgmXYh0fGErPjmPtBwKFO)
            
            ![picture](https://drive.google.com/uc?id=1NVJolPuD0yRrePuFLPLNES5gdOonujVv)
            
            1. 예를 들어 모델이 답한대로 “전자금융거래법 제49조 제4항 제2호” 위반의 건이라고 가정하면, 판사는 이에 대한 의무 규정인 “동법 제6조 제3항 제2호 또는 제3호”를 함께 명시했을 것임. 그러나 모델은 오직 벌칙 규정인 제49조 제4항 제2호만 명시했음.
            2. 또한 아동복지법 제71조 제1항 제2호 위반 시에도, 의무규정인 아동복지법 제17조 제3호를 명시했어야 함.
            
            ![picture](https://drive.google.com/uc?id=1Yq3nAUMGcRXbHEOOgbiMJCPulRUU7D9t)
            
        3. 나아가 train set이 형법, 각 특별법, 각 행정법 간 골고루 분포하고 있지 않기 때문에, 특히나 각 행정법은 training이 부족한 문제점도 가지고 있음. 
        train/valid/test 간의 데이터 분포 차이는 크지 않지만,
            
            ![picture](https://drive.google.com/uc?id=1YS3n-wcVpFdhftIFVgQRv2sIo9UhPpBi)
            
            1. Unknown 법률의 존재
            2. class imbalance로 인해 representation layer 자체가 제대로 학습되지 않았을 가능성이 큼


In [12]:
'''
Problem Law.1, Answer 2.D.c의 distribution plot 코드
주석 제거 후 실행하면 동일한 figure plot 및 저장됨
'''
#import matplotlib.pyplot as plt

def get_dist(split):
    statutes = ['Unknown'] + sorted(set.union(*map(set, data['train']['statutes'])))
    dist = [0] * len(statutes)
    for d in data[split]:
        for s in d['statutes']:
            dist[statutes.index(s) if s in statutes else 0] += 1
    
    return dist

def plot_dist(train_dist, valid_dist, test_dist):
    sts = sorted(range(len(train_dist)), key=lambda x: -train_dist[x])
    train_cnt = [train_dist[s] / sum(train_dist) for s in sts]
    valid_cnt = [valid_dist[s] / sum(valid_dist) for s in sts]
    test_cnt = [test_dist[s] / sum(test_dist) for s in sts]
    plt.figure(figsize=(20, 6))
    x = list(map(str, sts))
    plt.bar(x, train_cnt, color=(1, 0, 0, 0.7), label='train')
    plt.bar(x, valid_cnt, color=(0, 1, 0, 0.6), label='valid')
    plt.bar(x, test_cnt, color=(0, 0, 1, 0.5), label='test')
    plt.xticks(x, rotation=90, fontsize=8)
    plt.xlim(-1, len(sts))
    plt.legend()
    plt.savefig('dist.png', dpi=100)
    plt.show()

# train_dist = get_dist('train')
# valid_dist = get_dist('validation')
# test_dist = get_dist('test')
# plot_dist(train_dist, valid_dist, test_dist)

**Problem Law.2** *(10 points)*. 현재 모델은 법률의 내용을 모른체로 분류를 진행합니다. 모델이 법률의 내용을 알게 된다면 어떤 이점이 있을까요? 현재는 오답을 내고 있는 예제 중 법률 내용을 모델이 고려할 수 있다면 정답을 맞출 수도 있는 경우가 있나요?


1. 전술한 사안 중 2번, 즉 **법률 각 조항 간 관계성**을 고려한 경우 정답률이 상승할 것이라고 생각함.
2. 이를 위해서는 다음과 같은 내용을 train시켜야 함.
    1. “피고인은 &#126;&#126;와 공모하여”와 같이 공범성을 표현하는 문장이 있으면 형법 제30조~제34조의 공범 규정을 답하게 한다.
    2. 부작위범, 미수범 등에 대해서도 형법 제25조, 형법 제18조, 형법 제342조 등 각 죄의 미수범 처벌규정을 답하게 한다.
    3. 상상적 경합 관계의 죄 조합을 학습시킨다. 
        1. 무면허운전-음주운전
        2. 교통사고 시 차량 손괴-인적 상해
        3. 공무집행방해-준강도
        4. 사기-알선수뢰죄
        5. 강간, 강도 수단으로-감금죄
        등
    4. 특별법에 해당하는 사안은 특별법을 우선으로 답하게 한다.
        1. 1억 원 이상의 뇌물 관련 범죄인 경우
        2. 공동으로 집단 폭력행위가 있었던 경우
        3. 합동강간, 주거침입 이후 강간 등
        4. 아동을 대상으로 하는 범죄의 경우(이때 아동의 연령 고려)
        등
    5. 행정법은 벌칙규정-의무규정 쌍을 학습시킨다.
        1. 도로교통법 제148조의2 제1항 - 도로교통법 제44조 제1항 or 제2항
        2. 도로교통법 제148조의2 제3항 - 도로교통법 제44조 제3항
        등
3. + 범죄 사실 중 결론 문장에 더 큰 weight 부여
    1. 범죄사실의 내용이 긴 경우에는 모델에게 주목해야 하는 문장의 weight를 높여서 학습시킨다. 
    2. 범죄사실이 길어지면 마지막 결론 문장에 “결국 피고인은 ~하였다.” “이로써 피고인은 ~하였다.”와 같이 범죄의 구성요건을 요약하여 정리하는 경우가 많은데, 해당 문장의 weight를 높이면 정답률이 상승할 것이라고 생각한다.

![picture](https://drive.google.com/uc?id=1l-OxeIKIyTX8LBbopi1Ip3mhX3n5A6Cz)

![picture](https://drive.google.com/uc?id=1GnvBDg4OtPoeiatX7wloi-dWHeOgVxgs)

![picture](https://drive.google.com/uc?id=1gLfoX33KkbyGbJ8zNcWc6Itp9kUle9HF)


**Problem Law.3** *(10 points)*. 위 모델을 활용하여 (추가학습 없이) 유사 사실관계 검색을 하려 합니다. 어떻게 접근 할 수 있을지 간단하게 설명해 주세요.


- 모델에 사실관계를 넣으면 feature vector (e.g., 모델의 representation layer에서 추출한 encoding, 모델 마지막 레이어에서 softmax 함수를 적용하기 전의 logit) 를 얻을 수 있음
- 두 feature 간의 유사도 (e.g. cosine-similarity)  계산
- 두 사실관계로부터 모델을 통해 추출한 feature가 높은 유사도 값을 가진다면 두 사실관계가 유사할 가능성이 높음
- 검색하려는 사실관계를 입력받아 모델로 feature를 추출한 후, feature space 내에서 유사도가 높은 다른 사실관계들을 찾아내서 보여주면 됨

## *첨부: 정답이 잘못된 사안*

1. 특가법 5조의1조는 존재하지 않음
엘박스 확인결과 특가법 5조의10이 잘못 입력된 것으로 보임
    
    ![picture](https://drive.google.com/uc?id=1ulQFZylhp5uK4p9Z0aiLe4pgYtjC8jGh)
        
    ![picture](https://drive.google.com/uc?id=1TnQeY20oeyxevhEWGOGFx8JXopMaSPz-)
    
2. 형법 97조는 항, 호가 존재하지 않는 단일 조문임. 이 내용은 엘박스에도 잘못 등록되어 있는 것 같음.
전기통신 사업법의 몇조 몇호 위반인지 명시되어 있지 않고 형법과 섞여서 써 있음.
[https://lbox.kr/case/서울북부지방법원/2021고정475](https://lbox.kr/case/%EC%84%9C%EC%9A%B8%EB%B6%81%EB%B6%80%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90/2021%EA%B3%A0%EC%A0%95475)
    
    ![picture](https://drive.google.com/uc?id=16kHSjE7Wj2k44NLjKCllafyvaKL6BeJF)
    
    ![picture](https://drive.google.com/uc?id=1FSeNyDXz0dGDtrGBYc5ZPz9kLWl22oWl)
    
3. 교통사고 처리 특례법 제3조 제1항과 형법 268조인데, 교특법 268조라고 잘못 기재되어 있음
[https://lbox.kr/case/창원지방법원진주지원/2021고단1317](https://lbox.kr/case/%EC%B0%BD%EC%9B%90%EC%A7%80%EB%B0%A9%EB%B2%95%EC%9B%90%EC%A7%84%EC%A3%BC%EC%A7%80%EC%9B%90/2021%EA%B3%A0%EB%8B%A81317)
    
    ![picture](https://drive.google.com/uc?id=1gyf5IK20am6QuXNXC9_H4s7GEGjV2HIb)
    
    ![picture](https://drive.google.com/uc?id=1ZWmsC0IDhswTqdDo0CKaiNBBgcfApFUB)
    
4. 하나의 판결문에 2가지 범죄사실이 기재되어 있는 데이터인 경우
원칙적으로 본 모델은 1가지 범죄사실만을 찾도록 설계되어 있을 것이므로, 잘못된 출제된 문제라고 생각함
또한 판결문에서 “제314조 제1항”으로 기재될 내용이 “제314조”로만 기재되어 있음
    
    ![picture](https://drive.google.com/uc?id=1mlXJDE5pd9Kn2R_GGMLCfAFrAMrL4WGw)
    
    ![picture](https://drive.google.com/uc?id=1myrFPQ9QXM8EDVpG1gguUBuVNUClO65z)
    
    ![picture](https://drive.google.com/uc?id=1FTSXXRC6TpbYgfb72yDXbKg760mfYa8c)
    
5. 오류는 아니지만 마찬가지로 2개 범죄사실이 한 판결문에 동시에 들어있는 경우
사문서 위조죄-위조사문서 행사죄
    
    ![picture](https://drive.google.com/uc?id=17ua0YIBnVZlvWpMFPdXoCZb0nippm0uE)